In [157]:
import pandas as pd
import numpy as np
# import datetime
from datetime import datetime, timezone
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

In [158]:
def create_tuple(response):
    # all data
    response_data = response.json()
    # print(response_data)
    # data for a single target
    response_datum = response_data[0]
    # print(response_datum)
    # the target - e.g. "120MapleAve.STM.M22-V/AverageMassFlow"
    response_target = response_datum['target']
    # the metric for this target - e.g. "Steam"
    #response_metric = response_datum['metric']
    # print(response_metric)
    # the datapoints as a list of lists
    response_datapoints = response_datum['datapoints']
    # print(response_datapoints)
    # array of tuples for numpy
    tuple_array = [tuple(x) for x in response_datapoints]
    # make a numpy array
    npa = np.array(tuple_array, dtype=[
                   ('value', np.double), ('ts', 'datetime64[ms]')])
    # npa = npa.astype([('value', np.double), ('ts', datetime)])

    # print("here")
    # for index, i in enumerate(npa):
    #   j = pd.to_datetime(i[1]).replace(tzinfo=timezone.utc).astimezone(tz="America/New_York")
    #   npa[index][1] = j
    return npa

In [159]:
def current_date():
    return str(datetime.now())

In [160]:
"""
Returns an array of tuples where tuple[0] is the metric, and tuple[1] is the time 
of measurement

Target: a WebCTRL point name in the form of a string 
Example: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log"

Server: a string representing the WebCTRL server our point exists in
Example: "biotech_main"

Start: the start time we wish to be querying data about (in string format)
Example: "2021-12-31T11:00Z"

End: The end time we wish to be querying data about
Example: "2022-12-31T11:00Z"
"""
def fume_query(target,server, start,end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
          },
          "target": target
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)

In [161]:
"""
Returns a time series of the ithaca air temperature over the respective 
"""

def outside_temp(start,end):
  # https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query doesn't work
  # https://portal.emcs.cornell.edu/api/datasources/proxy/5/query works
  # https://portal-api.emcs.cucloud.net/query works
  url = "https://portal-api.emcs.cucloud.net/query"
  target = "GameFarmRoadWeatherStation.TAVG_H_F"
  data = {
    "range": {
        "from": start,
        "to": end
    },
    "targets": [
        {
            "target": target,
        }
    ]
  }
  request = requests.post(url, json=data)
  print(request)
#   print(request.json())
  return create_tuple(request)

In [162]:
from datetime import datetime
from dateutil import parser
import matplotlib.dates as mdates

def query(target_list, server, start, end):
  cfm_master = fume_query(target_list[0],server,start,end)
  sash_master = fume_query(target_list[3],server,start,end)
  occ_master = fume_query(target_list[2],server,start,end)
  internal_temp_master = fume_query(target_list[1],server,start,end)
  external_temp_master = outside_temp(start,end)
  print("\nExternal Temp Master ", external_temp_master)

  cfm_list = pd.Series(data=[i[0] for i in cfm_master], index=[i[1] for i in cfm_master])
  print("CFM List: ", cfm_list)
  cfm_list = cfm_list[~cfm_list.index.duplicated()]
  print("CFM List new: ", cfm_list)

  sash_list = pd.Series(data=[i[0] for i in sash_master], index=[i[1] for i in sash_master])
  print("\nSash List: ", sash_list)
  sash_list = sash_list[~sash_list.index.duplicated()]
  print("\nSash List new: ", sash_list)

  occ_list = pd.Series(data=[i[0] for i in occ_master], index=[i[1] for i in occ_master])
  print("\nOCC List: ", occ_list)
  occ_list = occ_list[~occ_list.index.duplicated()]
  print("\nOCC List new: ", occ_list)
  
  internal_temp_list = pd.Series(data=[i[0] for i in internal_temp_master], index=[i[1] for i in internal_temp_master])
  print("\nInternal Temp List: ", internal_temp_list)
  internal_temp_list = internal_temp_list[~internal_temp_list.index.duplicated()]
  print("\nInternal Temp List new: ", internal_temp_list)

  external_temp_list = pd.Series(data=[i[0] for i in external_temp_master], index=[i[1] for i in external_temp_master])
  print("\nExternal Temp List: ", external_temp_list)
  # external_temp_list = pd.read_csv("C:/Users/Dan/Documents/GitHub/fume-hood-dashboard/game_farm/hist.csv", index_col=0).squeeze()
  external_temp_list.index = external_temp_list.index.astype('datetime64[ns]')
  print("\nExternal Temp List: ", external_temp_list)

  # hourly data frame
  df = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list, external_temp_list], axis=1)
  df.columns = ["cfm", "sash" ,"occ", "internal temp", "external temp"]
  display(df)

  df["BTUh"] = 1.08 * df["cfm"] * (df["internal temp"] - df["external temp"])

  display(df)

In [163]:
def new_query(target_list, server, start, end):
  external_temp_master = outside_temp(start,end)
  print("\nExternal Temp Master ", external_temp_master)
  cfm_master = fume_query(target_list[0],server,start,end)
  sash_master = fume_query(target_list[3],server,start,end)
  occ_master = fume_query(target_list[2],server,start,end)
  internal_temp_master = fume_query(target_list[1],server,start,end)
  
  cfm_list = pd.Series(data=[i[0] for i in cfm_master], index=[i[1] for i in cfm_master])
  print("CFM List: ", cfm_list)
  cfm_list = cfm_list[~cfm_list.index.duplicated()]
  print("CFM List new: ", cfm_list)

  sash_list = pd.Series(data=[i[0] for i in sash_master], index=[i[1] for i in sash_master])
  print("\nSash List: ", sash_list)
  sash_list = sash_list[~sash_list.index.duplicated()]
  print("\nSash List new: ", sash_list)

  occ_list = pd.Series(data=[i[0] for i in occ_master], index=[i[1] for i in occ_master])
  print("\nOCC List: ", occ_list)
  occ_list = occ_list[~occ_list.index.duplicated()]
  print("\nOCC List new: ", occ_list)
  
  internal_temp_list = pd.Series(data=[i[0] for i in internal_temp_master], index=[i[1] for i in internal_temp_master])
  print("\nInternal Temp List: ", internal_temp_list)
  internal_temp_list = internal_temp_list[~internal_temp_list.index.duplicated()]
  print("\nInternal Temp List new: ", internal_temp_list)

  external_temp_list = pd.Series(data=[i[0] for i in external_temp_master], index=[i[1] for i in external_temp_master])
  print("\nExternal Temp List: ", external_temp_list)
  # external_temp_list = pd.read_csv("C:/Users/Dan/Documents/GitHub/fume-hood-dashboard/game_farm/hist.csv", index_col=0).squeeze()
  external_temp_list.index = external_temp_list.index.astype('datetime64[ns]')
  print("\nExternal Temp List: ", external_temp_list)

  # interpolating External Temperature values
  ext_temp_hr = external_temp_list.tolist()
  ext_temp = []
  i = 0
  while i < (len(ext_temp_hr)-1):
    initial = ext_temp_hr[i]
    difference = (ext_temp_hr[i+1] - initial)/12
    ext_temp_5 = []
    for x in range(12):
      ext_temp_5.append(initial + difference*x)
    print("\n 12 Ext Temp Values: ", ext_temp_5)
    ext_temp = ext_temp + ext_temp_5
    i += 1
  ext_temp = ext_temp[12:] + [ext_temp_hr[-1]]
  print("\nList of New Ext Temp Values: ", ext_temp)

  # Internal Temperatures span multiple days, External Temperatures span one day
  new_external_temp_list = pd.Series(data=ext_temp, index=[i[1] for i in internal_temp_master[:277]])
  print("\nNew External Temp List: ", new_external_temp_list)

  new_df = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list, new_external_temp_list], axis=1)
  new_df.columns = ["cfm", "sash" ,"occ", "internal temp", "external temp"]
  display(new_df)
  new_df = new_df.dropna()
  new_df["BTUh"] = 1.08 * new_df["cfm"] * (new_df["internal temp"] - new_df["external temp"])

  display(new_df)

In [164]:
target_list = ["#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash"]
server = "biotech_main"
start = str(datetime(2022, 11, 17))
end = str(datetime(2022, 11, 18))

#query(target_list,server,start,end)
new_query(target_list,server,start,end)

<Response [200]>

External Temp Master  [(37.6286, '2022-11-17T00:00:00.000') (37.1696, '2022-11-17T01:00:00.000')
 (36.59  , '2022-11-17T02:00:00.000') (36.203 , '2022-11-17T03:00:00.000')
 (36.1958, '2022-11-17T04:00:00.000') (35.9762, '2022-11-17T05:00:00.000')
 (35.384 , '2022-11-17T06:00:00.000') (34.8314, '2022-11-17T07:00:00.000')
 (34.5938, '2022-11-17T08:00:00.000') (34.2158, '2022-11-17T09:00:00.000')
 (33.4256, '2022-11-17T10:00:00.000') (32.3528, '2022-11-17T11:00:00.000')
 (32.45  , '2022-11-17T12:00:00.000') (33.1556, '2022-11-17T13:00:00.000')
 (34.4642, '2022-11-17T14:00:00.000') (34.862 , '2022-11-17T15:00:00.000')
 (32.0828, '2022-11-17T16:00:00.000') (31.8758, '2022-11-17T17:00:00.000')
 (34.0178, '2022-11-17T18:00:00.000') (31.9406, '2022-11-17T19:00:00.000')
 (33.9044, '2022-11-17T20:00:00.000') (34.1906, '2022-11-17T21:00:00.000')
 (33.0026, '2022-11-17T22:00:00.000') (32.8064, '2022-11-17T23:00:00.000')
 (32.4374, '2022-11-18T00:00:00.000')]
<Response [200]>
<Res

,cfm,sash,occ,internal temp,external temp
2022-11-17 05:00:00,224.13,1.2,1.0,72.2,37.1696
2022-11-17 05:05:00,224.13,1.2,1.0,72.4,37.1213
2022-11-17 05:10:00,224.13,1.2,1.0,72.4,37.0730
2022-11-17 05:15:00,225.10,1.2,1.0,72.1,37.0247
2022-11-17 05:20:00,225.10,1.2,1.0,71.8,36.9764
...,...,...,...,...,...
2022-11-19 04:35:00,224.13,1.2,0.0,70.3,NaN
2022-11-19 04:40:00,225.59,1.2,0.0,70.3,NaN
2022-11-19 04:45:00,227.06,1.2,0.0,70.2,NaN
2022-11-19 04:50:00,227.06,1.2,0.0,70.2,NaN


,cfm,sash,occ,internal temp,external temp,BTUh
2022-11-17 05:00:00,224.13,1.2,1.0,72.2,37.16960,8479.472636
2022-11-17 05:05:00,224.13,1.2,1.0,72.4,37.12130,8539.576233
2022-11-17 05:10:00,224.13,1.2,1.0,72.4,37.07300,8551.267751
2022-11-17 05:15:00,225.10,1.2,1.0,72.1,37.02470,8527.086032
2022-11-17 05:20:00,225.10,1.2,1.0,71.8,36.97640,8465.895749
...,...,...,...,...,...,...
2022-11-18 03:40:00,224.13,2.2,0.0,70.9,32.56040,9280.498912
2022-11-18 03:45:00,223.64,2.2,0.0,70.9,32.52965,9267.636680
2022-11-18 03:50:00,224.62,2.2,0.0,70.8,32.49890,9291.448529
2022-11-18 03:55:00,220.22,2.2,0.0,70.8,32.46815,9116.755208
